## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zyryanka,RU,65.7500,150.8500,32.23,66,98,9.31,overcast clouds
1,1,Cape Town,ZA,-33.9258,18.4232,52.12,75,39,3.09,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,53.17,65,87,19.22,light rain
3,3,Nalut,LY,30.3333,10.8500,70.38,37,0,8.68,clear sky
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,81,40,8.05,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input('What is the minimum temperature you would like for your trip?'))
max_temp=float(input('What is the maximum temperature you wold like for your trip?'))

What is the minimum temperature you would like for your trip?65
What is the maximum temperature you wold like for your trip?70


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Santa Marinella,IT,42.0345,11.8536,66.69,82,5,12.12,clear sky
33,33,Kindu,CD,-2.9500,25.9500,69.44,100,100,2.51,moderate rain
59,59,Marsaxlokk,MT,35.8419,14.5431,65.01,88,75,11.50,thunderstorm
82,82,Ierapetra,GR,35.0100,25.7353,68.18,62,2,13.00,clear sky
94,94,Colac,AU,-38.3333,143.5833,69.15,79,95,9.40,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                60
City                   60
Country                60
Lat                    60
Lng                    60
Max Temp               60
Humidity               60
Cloudiness             60
Wind Speed             60
Current Description    60
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No null rows

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
#hotel_df.head(10)
hotel_df.count()

City                   60
Country                60
Max Temp               60
Current Description    60
Lat                    60
Lng                    60
Hotel Name             60
dtype: int64

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.......Skipping.")
       

Hotel not found.......Skipping.
Hotel not found.......Skipping.
Hotel not found.......Skipping.
Hotel not found.......Skipping.
Hotel not found.......Skipping.
Hotel not found.......Skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace("",np.nan,inplace=True)
hotel_df.dropna(axis=0, how='any', inplace=True)

In [11]:
# 8a. Create the output File (CSV)
output_data_file="..\Vacation_Search\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))